https://konlpy.org/_/downloads/en/latest/pdf/ 참고

>- morphs : 형태소 추출
>- pos : 품사 태깅(Part-of-speech tagging)
>- nouns : 명사 추출
>- sentences : 문장 추출

# Time analysis
1. Loading Time : class loading time , including dictionary loads
    - Kkma : 5.6988초
    - Komoran : 5.4866초
    - Hannanum : 0.6591초
    - Okt : 1.4870초
    - Mecab : 0.0007초
        - 확연하게 라이브러리 별로 로딩시간이 존재하는것을 볼 수 있다.
2. Execution Time : Time for executing the pos method for each class , with 100K characters
    - Kkma : 35.6초
    - Komoran : 25.6초
    - Hannanum : 8.8초
    - Okt : 2.47초
    - Mecab : 0.283초
        - 실행 시간또한 라이브러리에 따라 달라진다 . 비교적 Mecab이 빠른 편이다
   

# Finding collocations 

>- collocation : 연어(어떤 언어 내에서 특정한 뜻을 나타낼 때 흔히 함께 쓰이는 단어들의 결합)


In [16]:
from konlpy.tag import Kkma
from konlpy.corpus import kolaw
from konlpy.utils import pprint
from nltk import collocations

measures = collocations.BigramAssocMeasures()


#koloaw 라는 대한민국 헌법 관련 텍스트를 불러오는 과정
doc = kolaw.open('constitution.txt').read()

print('\nCollocations among tagged words:')
tagged_words = Kkma().pos(doc)
finder = collocations.BigramCollocationFinder.from_words(tagged_words)
pprint(finder.nbest(measures.pmi, 10)) # top 5 n-grams with highest PMI

print('\nCollocations among words:')
words = [w for w, t in tagged_words]
ignored_words = [u'안녕']
finder = collocations.BigramCollocationFinder.from_words(words)
finder.apply_word_filter(lambda w: len(w) < 2 or w in ignored_words)
finder.apply_freq_filter(3) # only bigrams that appear 3+ times
pprint(finder.nbest(measures.pmi, 10))

print('\nCollocations among tags:')
tags = [t for w, t in tagged_words]
finder = collocations.BigramCollocationFinder.from_words(tags)
pprint(finder.nbest(measures.pmi, 5))



Collocations among tagged words:
[(('가부', 'NNG'), ('동수', 'NNG')),
 (('강제', 'NNG'), ('노역', 'NNG')),
 (('경자', 'NNG'), ('유전', 'NNG')),
 (('고', 'ECS'), ('채취', 'NNG')),
 (('공무', 'NNG'), ('담임', 'NNG')),
 (('공중', 'NNG'), ('도덕', 'NNG')),
 (('과반', 'NNG'), ('수가', 'NNG')),
 (('교전', 'NNG'), ('상태', 'NNG')),
 (('그러', 'VV'), ('나', 'ECE')),
 (('기본적', 'NNG'), ('인권', 'NNG'))]

Collocations among words:
[('현행', '범인'),
 ('형의', '선고'),
 ('내부', '규율'),
 ('정치적', '중립성'),
 ('누구', '든지'),
 ('회계', '연도'),
 ('지체', '없이'),
 ('평화적', '통일'),
 ('형사', '피고인'),
 ('지방', '자치')]

Collocations among tags:
[('XR', 'XSA'), ('JKC', 'VCN'), ('EPT', 'EPT'), ('VCN', 'ECD'), ('ECD', 'VX')]


# collocation(연어) 활용 예제 

In [38]:
text = '동해 물과 백두산이 마르고 닳도록\
    하느님이 보우하사 우리나라 만세\
    남산 위에 저 소나무 철갑을 두른 듯\
    바람서리 불변함은 우리 기상일세\
    가을 하늘 공활한데 높고 구름 없이\
    밝은 달은 우리 가슴 일편단심일세\
    이 기상과 이 맘으로 충성을 다하여\
    괴로우나 즐거우나 나라 사랑하세'

print('\nCollocations among tagged words:')
tagged_words = Kkma().pos(text)
finder = collocations.BigramCollocationFinder.from_words(tagged_words)
pprint(finder.nbest(measures.pmi, 10)) # top 5 n-grams with highest PMI

print('\nCollocations among words:')
words = [w for w, t in tagged_words]
finder = collocations.BigramCollocationFinder.from_words(words)
finder.apply_word_filter(lambda w: len(w) < 2)
finder.apply_freq_filter(1) # 한번이라도 나온 것들로?
pprint(finder.nbest(measures.pmi, 10))

print('\nCollocations among tags:')
tags = [t for w, t in tagged_words]
finder = collocations.BigramCollocationFinder.from_words(tags)
pprint(finder.nbest(measures.pmi, 5))



Collocations among tagged words:
[(('가슴', 'NNG'), ('일편단심', 'NNG')),
 (('가을', 'NNG'), ('하늘', 'NNG')),
 (('공', 'NNG'), ('활', 'NNG')),
 (('과', 'JC'), ('백두산', 'NNP')),
 (('구름', 'NNG'), ('없이', 'MAG')),
 (('기상', 'NNG'), ('과', 'JKM')),
 (('기상일', 'NNG'), ('세', 'MDN')),
 (('나라', 'NNG'), ('사랑', 'NNG')),
 (('남산', 'NNP'), ('위', 'NNG')),
 (('다하', 'VV'), ('여', 'ECS'))]

Collocations among words:
[('가슴', '일편단심'),
 ('가을', '하늘'),
 ('구름', '없이'),
 ('나라', '사랑'),
 ('도록', '하느님'),
 ('만세', '남산'),
 ('바람', '서리'),
 ('보우', '하사'),
 ('사랑', '하세'),
 ('서리', '불변')]

Collocations among tags:
[('VCP', 'EFN'), ('EFN', 'MDT'), ('ETN', 'JX'), ('JC', 'NNP'), ('JX', 'NP')]


# Chunking 
>- 단기기억의 제한된 용량을 극복하는 방법의 하나로 정보의 개별적 단위를 보다 큰 의미 있는 단위로 묶는 것이다.
>- 정보는 크기가 아닌 단위 수가 작업기억의 한계와 관련되므로 각각의 정보를 큰 단위로 묶으면 더 많은 정보를 보유할 수 있게 된다.
>- 전화번호를 국번과 번호로 나누는 것도 기억을 돕는 청킹의 전략이다.

In [47]:
import konlpy
import nltk
# POS tag a sentence
sentence = u'만 6세 이하의 초등학교 취학 전 자녀를 양육하기 위해서는'
words = konlpy.tag.Okt().pos(sentence)

# Define a chunk grammar, or chunking rules, then chunk
grammar = """
NP: {<N.*>*<Suffix>?} # Noun phrase
VP: {<V.*>*}          # Verb phrase
AP: {<A.*>*}          # Adjective phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)
        
# Display the chunk tree
chunks.draw()

# KONLPY 대표 라이브러리
- 1.Hannanum
- 2.Kkma
- 3.Komoran
- 4.Mecab
- 5.Okt

## 1. Hannanum Class 

In [63]:
from konlpy.tag import Hannanum
hannanum = Hannanum()
print(hannanum.analyze('롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'),'\n\n')
print(hannanum.nouns('롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'),'\n\n')
print(hannanum.nouns('다람쥐 헌 쳇바퀴에 타고파'),'\n\n')
print(hannanum.pos('웃으면 더 행복합니다!'))

[[[('롯데마트', 'ncn'), ('의', 'jcm')], [('롯데마트의', 'ncn')], [('롯데마트', 'nqq'), ('의', 'jcm')], [('롯데마트의', 'nqq')]], [[('흑마늘', 'ncn')], [('흑마늘', 'nqq')]], [[('양념', 'ncn')]], [[('치킨', 'ncn'), ('이', 'jcc')], [('치킨', 'ncn'), ('이', 'jcs')], [('치킨', 'ncn'), ('이', 'ncn')]], [[('논란', 'ncpa'), ('이', 'jcc')], [('논란', 'ncpa'), ('이', 'jcs')], [('논란', 'ncpa'), ('이', 'ncn')]], [[('되', 'nbu'), ('고', 'jcj')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecc')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecs')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecx')], [('되', 'paa'), ('고', 'ecc')], [('되', 'paa'), ('고', 'ecs')], [('되', 'paa'), ('고', 'ecx')], [('되', 'pvg'), ('고', 'ecc')], [('되', 'pvg'), ('고', 'ecs')], [('되', 'pvg'), ('고', 'ecx')], [('되', 'px'), ('고', 'ecc')], [('되', 'px'), ('고', 'ecs')], [('되', 'px'), ('고', 'ecx')]], [[('있', 'paa'), ('다', 'ef')], [('있', 'px'), ('다', 'ef')]], [[('.', 'sf')], [('.', 'sy')]]] 


['롯데마트', '흑마늘', '양념', '치킨', '논란'] 


['다람쥐', '쳇바퀴', '타고파'] 


[('웃', 'P'), ('으면', 'E'), ('더', 'M'), ('행복', 'N'), ('하', 

## 2.Kkma Class

In [66]:
from konlpy.tag import Kkma
kkma = Kkma()
print(kkma.morphs('공부를 하면 할수록 모르는게 많다는 것을 알게 됩니다.'),'\n\n')
print(kkma.nouns('동해물과 백두산이 마르고 닳도록 ...'),'\n\n')
print(kkma.pos('하느님이 보우하사 우리 나라 만세'),'\n\n')
print(kkma.sentences('무궁화 삼천리 화려강산이다 . 대한사람 대한으로 길이 보전하자!'))

['공부', '를', '하', '면', '하', 'ㄹ수록', '모르', '는', '것', '이', '많', '다는', '것', '을', '알', '게', '되', 'ㅂ니다', '.'] 


['물', '백두산'] 


[('하느님', 'NNG'), ('이', 'JKS'), ('보우', 'NNG'), ('하사', 'NNG'), ('우리', 'NP'), ('나라', 'NNG'), ('만세', 'NNG')] 


['무궁화 삼천리 화려 강산이다 .', '대한 사람 대한으로 길이 보전하자!']


## 3. Komoran 

In [72]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='/tmp/dic.txt')
print(komoran.morphs('우왕 코모란도 오픈소스가 되었어요'),'\n\n')
print(komoran.nouns('오픈소스에 관심 많은 멋진 개발자님들!'),'\n\n')
print(komoran.pos('혹시 바람과 함께 사라지다 봄?'))

['우왕', '코', '모란', '도', '오픈', '소스', '가', '되', '었', '어요'] 


['오픈', '소스', '관심', '개발자'] 


[('혹시', 'MAG'), ('바람과 함께 사라지다', 'NNP'), ('봄', 'NNG'), ('?', 'SF')] 




## 4. Mecab

- Mecab은 윈도우에서는 사용이 안된다고 함

##  5. Okt
- 과거에는 Twitter라는 모듈로 사용되었다고 함

In [74]:
from konlpy.tag import Okt
okt = Okt()
print(okt.morphs(u'단독입찰보다 복수입찰의 경우'))
print(okt.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))
print(okt.phrases(u'날카로운 분석과 신뢰감 있는 진행으로'))
print(okt.pos(u'이것도 되나욬ㅋㅋ'))
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True))
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True))

['단독', '입찰', '보다', '복수', '입찰', '의', '경우']
['항공기', '체계', '종합', '개발', '경험']
['날카로운 분석', '날카로운 분석과 신뢰감', '날카로운 분석과 신뢰감 있는 진행', '분석', '신뢰', '진행']
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나욬', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나요', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]
